In [1]:
import os
import pandas as pd

class Reader:
    def __init__(self, filePath):
        newses=[]
        news=pd.read_csv(filePath,encoding="UTF-8")
        newses.append(news)
        self.docs=pd.concat(newses,ignore_index=True)
    
    def read(self, id):
        if id==0:
            return 0
        return str(self.docs['text'][id-1])

reader = Reader('./201901.csv')

print(reader.docs)

             date press category                             headline  \
0      2019/01/01   한겨레       정치     문희상 “임시정부 100년…새 역사 시작하는 새해 될 것”   
1      2019/01/01  세계일보    지역,경제                  새해에도 암울한 대구·경북 지역경제   
2      2019/01/01  세계일보    경제,지역            경남 승강기안전공단 지역과 상생 동반성장 활동   
3      2019/01/01  중앙일보       정치   문 대통령 신년사 “한 분 한 분의 삶 나아지도록 노력하겠다”   
4      2019/01/01  동아일보       경제       이마트, 올해 상반기 ‘한우 등심’ 초저가로 가격 동결   
...           ...   ...      ...                                  ...   
24467  2019/01/31  중앙일보    국제,정치        비핵화 협상 기로…뭉치는 북방3각, 멀어지는 남방3각   
24468  2019/01/31  중앙일보    국제,정치          일본서 여성의원·활동가에 '강매 괴롭힘'…수법보니   
24469  2019/01/31  국민일보    사회,지역    숨진 아내 몸에 ‘방어흔’… 이틀 만에 발견된 노부부의 비극   
24470  2019/01/31  서울신문    국제,정치  “트럼프·시진핑, 북미정상회담 후 2월 말 정상회담 개최 논의”   
24471  2019/01/31  내일신문       사회       자기주도학습 중심, 일산 지역 독학재수학원을 소개합니다   

                                                     url  \
0      http://www.hani.co.kr/arti/politics/politics_g...   
1  

In [5]:
import sys
import io
import re # 정규표현식 패키지
from kiwipiepy import Kiwi, Option

file = open("한국어불용어100.txt", 'r', encoding="utf-8")
stopword=[]

while True:
    line = file.readline()
    if not line: break
    wordlist = line.split('\t')
    if (wordlist[1].startswith('N')):
        stopword.append(wordlist[0])


kiwi=Kiwi()
kiwi.extract_add_words(reader.read)
kiwi.prepare()

print(stopword)

['것', '수', '이', '나', '사람', '등', '우리', '때', '년', '말', '일', '때문', '그것', '일', '문제', '사회', '중', '씨', '지금', '속', '하나', '집', '월', '데', '자신', '내', '경우', '명', '생각', '시간', '그녀', '앞', '번', '여자', '개', '전', '사실', '점', '정도', '원', '소리']


In [6]:
stopwords = set(stopword)

def tokenize(sent): # 파일의 라인을 분석할 tokenize 함수
    res, score = kiwi.analyze(sent)[0] # 첫번째 결과를 사용한다, 분석할때 나오는 결과에서 단어만 추출
    return [word
            for word, tag, _, _ in res
            if tag.startswith('N') or tag.startswith("VA") and word not in stopwords] #불용어사전 적용

In [45]:
import tomotopy as tp


#LDA 모델을 적용해서 토픽 추출, k는 topic 개수
#alpha는 문헌-토픽 디리클레 분포의 하이퍼 파라미터
#eta는 토픽-단어 디리클레 분포의 하이퍼 파라미터 두개다 상수인듯하다
#min_cf는 단어의 최소 장서 빈도수, 전체 문헌내 출현빈도
#min_df는 단어의 최서 문헌 빈도수, 출현한 문헌 숫자수 의미
#tw는 용어 가중치 기법으로, ONE, IDF, PMI를 사용가능, ONE 보다는 PMI나 IDF 둘중 하나 사용 

for i in range(len(reader.docs)): #해당 경로의 파일을 받아와 한 라인씩 model에 추가
    model = tp.LDAModel(k=1, alpha=0.1, eta=0.01, min_cf=3,min_df=1, tw=tp.TermWeight.PMI)
    model.add_doc(tokenize(reader.docs['text'][i])) #추출하고 모델안의 문헌을 넣는다. 즉, 학습과정에 쓰일 문헌을 생성

    model.train(0) #학습 초기화

 
    for i in range(0,100):
        model.train(20) #문헌 학습, 안에 숫자는 깁스 샘플링의 반복횟수
		#이때 기본값으로 시스템내 가용한 모든 스레드의 개수사용, 그리고 병렬화 방법을 찾아서 실행시켜준다
  
    for i in range(model.k): #k는 토픽의 개수
        res = model.get_topic_words(i, top_n=10) #하위 토픽 i에 해당하는 top_n개의 단어 반환
        print('Topic #{}'.format(i), end='\t')
        print(', '.join(w for w, p in res))

Topic #0	경제, 미국, 기업, 세계, 상위, 년, 정치, 것, 트럼프, 부자
Topic #0	발굴, 유해, 년, 전사자, 아버지, 구, 전쟁, 월, 수, 만
Topic #0	당, 의원, 한국, 대표, 투쟁, 황, 것, 통합, 지도부, 국회
Topic #0	물가, 년, 상승, 전년, 대비, 가격, 등, 하락, 품목, 2019년
Topic #0	입당, 등, 일, 선거, 법, 연령, 고등학생, 예비, 대상, 당원
Topic #0	의원, 직, 없, 한국, 당, 총사퇴, 대표, 국민, 의장, 사퇴
Topic #0	P2P, 많, 자, 연체, 말, 기간, 계좌, 금융, 대출, 업체
Topic #0	대통령, 것, 국회, 법, 문, 야당, 국민, 군소, 편, 지지자
Topic #0	전셋값, 부동산, 수, 공급, 수요, 집값, 대책, 서울, 것, 정부
Topic #0	공무원, 국민, 사회, 경제, 규제, 공직, 새롭, 년, 성장, 정책
Topic #0	군사, 장관, 안보, 일, 군, 평화, 정, 미사일, 훈련, 태세
Topic #0	국회, 정치, 대한민국, 국민, 의장, 일, 년, 새롭, 문, 새해
Topic #0	공제, 신문, 소득, 년, 구독료, 혜택


KeyboardInterrupt: 